In [2]:
import pandas as pd

In [3]:
dd_sales = pd.read_csv('SALES_viewByOrder_2025-06-22_2025-08-22_Kb0yU_2025-08-28T17-46-06Z.csv')

dd_fin_detailed = pd.read_csv('financial_2025-06-22_2025-08-22_SJnhV_2025-08-28T17-45-19Z/FINANCIAL_DETAILED_TRANSACTIONS_2025-06-22_2025-08-22_SJnhV_2025-08-28T17-45-19Z.csv')

dd_fin_error = pd.read_csv('financial_2025-06-22_2025-08-22_SJnhV_2025-08-28T17-45-19Z/FINANCIAL_ERROR_CHARGES_AND_ADJUSTMENTS_2025-06-22_2025-08-22_SJnhV_2025-08-28T17-45-19Z.csv')

dd_fin_payout = pd.read_csv('financial_2025-06-22_2025-08-22_SJnhV_2025-08-28T17-45-19Z/FINANCIAL_PAYOUT_SUMMARY_2025-06-22_2025-08-22_SJnhV_2025-08-28T17-45-19Z.csv')

dd_fin_simplified = pd.read_csv('financial_2025-06-22_2025-08-22_SJnhV_2025-08-28T17-45-19Z/FINANCIAL_SIMPLIFIED_TRANSACTIONS_2025-06-22_2025-08-22_SJnhV_2025-08-28T17-45-19Z.csv')

dd_marketing_promo = pd.read_csv('marketing_2025-06-22_2025-08-22_FLGlw_2025-08-28T17-46-33Z/MARKETING_PROMOTION_2025-06-22_2025-08-22_FLGlw_2025-08-28T17-46-33Z.csv')

dd_marketing_sponsered = pd.read_csv('marketing_2025-06-22_2025-08-22_FLGlw_2025-08-28T17-46-33Z/MARKETING_SPONSORED_LISTING_2025-06-22_2025-08-22_FLGlw_2025-08-28T17-46-33Z.csv')


In [7]:
import pandas as pd

# Filter dd_fin_detailed for required transaction types
filtered_detailed = dd_fin_detailed[
    dd_fin_detailed["Transaction type"].isin(["Error Charge", "Adjustment"])
]

# Reset index for fair comparison
filtered_detailed = filtered_detailed.reset_index(drop=True)
dd_fin_error = dd_fin_error.reset_index(drop=True)

# Step 1: Check row counts
print("Row counts:")
print("Filtered Detailed:", len(filtered_detailed))
print("Error DF:", len(dd_fin_error))

# Step 2: Check if completely equal
if filtered_detailed.equals(dd_fin_error):
    print("✅ Both DataFrames match perfectly (same rows & content).")
else:
    print("❌ DataFrames do not match completely.")

    # Find mismatched rows
    diff1 = filtered_detailed.merge(dd_fin_error, how="outer", indicator=True)
    only_in_detailed = diff1[diff1["_merge"] == "left_only"]
    only_in_error = diff1[diff1["_merge"] == "right_only"]

    print("\nRows only in detailed (filtered):")
    print(only_in_detailed)

    print("\nRows only in error df:")
    print(only_in_error)


Row counts:
Filtered Detailed: 3562
Error DF: 3562
❌ DataFrames do not match completely.

Rows only in detailed (filtered):
Empty DataFrame
Columns: [Timestamp UTC time, Timestamp UTC date, Timestamp local time, Timestamp local date, Order received local time, Order pickup local time, Payout time, Payout date, Business ID, Business name, Store ID, Store name, Merchant store ID, Transaction type, Delivery UUID, DoorDash transaction ID, DoorDash order ID, Merchant delivery ID, POS order ID, Channel, Description, Final order status, Currency, Subtotal, Subtotal tax passed to merchant, Commission, Payment processing fee, Marketing fees | (including any applicable taxes), Customer discounts from marketing | (funded by you), Customer discounts from marketing | (funded by DoorDash), Customer discounts from marketing | (funded by a third-party), DoorDash marketing credit, Third-party contribution, Error charges, Adjustments, Net total, Pre-adjusted subtotal, Pre-adjusted tax subtotal, Subtotal

In [8]:
print("===== Filtered Detailed (head) =====")
print(filtered_detailed.head())
print("\nShape:", filtered_detailed.shape)
print("Columns:", len(filtered_detailed.columns))

print("\n===== Error DF (head) =====")
print(dd_fin_error.head())
print("\nShape:", dd_fin_error.shape)
print("Columns:", len(dd_fin_error.columns))


===== Filtered Detailed (head) =====
           Timestamp UTC time Timestamp UTC date        Timestamp local time  \
0  2025-08-22 20:38:39.470234         2025-08-22  2025-08-22 16:38:39.470234   
1  2025-08-22 19:26:22.014026         2025-08-22  2025-08-22 15:26:22.014026   
2  2025-08-22 17:01:14.263585         2025-08-22  2025-08-22 13:01:14.263585   
3  2025-08-22 11:49:21.431326         2025-08-22  2025-08-22 07:49:21.431326   
4  2025-08-22 04:09:04.002913         2025-08-22  2025-08-22 00:09:04.002913   

  Timestamp local date Order received local time     Order pickup local time  \
0           2025-08-22   2025-08-22 16:10:35.146  2025-08-22 16:26:07.499228   
1           2025-08-22   2025-08-20 21:03:36.515                         NaN   
2           2025-08-22    2025-08-22 12:42:22.35  2025-08-22 12:52:24.793994   
3           2025-08-22   2025-08-22 01:08:23.477  2025-08-22 01:21:36.586605   
4           2025-08-22   2025-08-21 01:49:50.712  2025-08-21 01:57:38.065157   

 

In [9]:
# --- Normalize columns ---
common_cols = list(set(filtered_detailed.columns) & set(dd_fin_error.columns))

fd_norm = filtered_detailed[common_cols].copy()
error_norm = dd_fin_error[common_cols].copy()

# Ensure same column order
fd_norm = fd_norm.sort_index(axis=1)
error_norm = error_norm.sort_index(axis=1)

# Reset index
fd_norm = fd_norm.reset_index(drop=True)
error_norm = error_norm.reset_index(drop=True)

# --- Check equality ---
print("\nRow counts match? ->", fd_norm.shape[0] == error_norm.shape[0])
print("Column counts match after normalization? ->", fd_norm.shape[1] == error_norm.shape[1])

if fd_norm.equals(error_norm):
    print("✅ DataFrames match perfectly after aligning common columns.")
else:
    print("❌ Still differences remain.")

    # Find element-wise differences
    diff_mask = fd_norm.ne(error_norm)

    # Count differences
    print("Number of differing cells:", diff_mask.sum().sum())

    # Show first 10 mismatches
    rows, cols = diff_mask.to_numpy().nonzero()
    for r, c in zip(rows[:10], cols[:10]):
        col = fd_norm.columns[c]
        print(f"Row {r}, Column {col} -> Detailed: {fd_norm.iat[r,c]} | ErrorDF: {error_norm.iat[r,c]}")



Row counts match? -> True
Column counts match after normalization? -> True
✅ DataFrames match perfectly after aligning common columns.


In [10]:
# Aggressive normalization
fd_clean = fd_norm.fillna("").astype(str).apply(lambda x: x.str.strip())
error_clean = error_norm.fillna("").astype(str).apply(lambda x: x.str.strip())

if fd_clean.equals(error_clean):
    print("✅ Perfect match after aggressive normalization (ignoring NaN/whitespace/datatype issues).")
else:
    print("❌ Still mismatches after aggressive cleaning.")


✅ Perfect match after aggressive normalization (ignoring NaN/whitespace/datatype issues).


In [13]:
dd_fin_simplified.describe

<bound method NDFrame.describe of        Business ID Business name  Store ID  \
0             5579    McDonald's    685609   
1             5579    McDonald's    685609   
2             5579    McDonald's    685609   
3             5579    McDonald's    685609   
4             5579    McDonald's    685609   
...            ...           ...       ...   
50326         5579    McDonald's  24590476   
50327         5579    McDonald's  24590476   
50328         5579    McDonald's  24590476   
50329         5579    McDonald's  24590476   
50330         5579    McDonald's  24590476   

                                Store name        Timestamp local time  \
0                McDonald's (16255-LORAIN)  2025-08-22 23:50:56.883777   
1                McDonald's (16255-LORAIN)  2025-08-22 23:50:41.467725   
2                McDonald's (16255-LORAIN)  2025-08-22 23:12:17.152464   
3                McDonald's (16255-LORAIN)  2025-08-22 23:09:35.432636   
4                McDonald's (16255-LORAIN) 

In [14]:
dd_fin_detailed.describe

<bound method NDFrame.describe of                Timestamp UTC time Timestamp UTC date  \
0      2025-08-23 03:50:56.883777         2025-08-23   
1      2025-08-23 03:50:41.467725         2025-08-23   
2      2025-08-23 03:12:17.152464         2025-08-23   
3      2025-08-23 03:09:35.432636         2025-08-23   
4      2025-08-23 03:09:24.582411         2025-08-23   
...                           ...                ...   
50326  2025-06-22 14:33:31.722204         2025-06-22   
50327  2025-06-22 13:56:13.328558         2025-06-22   
50328  2025-06-22 13:42:40.443785         2025-06-22   
50329  2025-06-22 13:41:24.748018         2025-06-22   
50330  2025-06-22 10:52:49.342203         2025-06-22   

             Timestamp local time Timestamp local date  \
0      2025-08-22 23:50:56.883777           2025-08-22   
1      2025-08-22 23:50:41.467725           2025-08-22   
2      2025-08-22 23:12:17.152464           2025-08-22   
3      2025-08-22 23:09:35.432636           2025-08-22   
4  